In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Libraries

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub

from PIL import Image
import matplotlib.pyplot as plt

# Load Data

In [ ]:
art_data_path = "/kaggle/input/best-artworks-of-all-time/images/images/"
wonders_data_path = "/kaggle/input/wonders-of-the-world-image-classification/Wonders of World/Wonders of World/"

In [ ]:
arts = []
artists = []
for artist in os.listdir(art_data_path):
    files = os.listdir(os.path.join(art_data_path, artist))
    for file in files:
        artists.append(artist)
        arts.append(os.path.join(art_data_path, artist, file))

In [ ]:
art_df = pd.DataFrame({
    "artist": artists,
    "art": arts,
})
art_df.head()

In [ ]:
art_df.shape

In [ ]:
wonders = []
wonders_files = []

for wonder in os.listdir(wonders_data_path):
    files = os.listdir(os.path.join(wonders_data_path, wonder))
    for file in files:
        wonders.append(wonder)
        wonders_files.append(os.path.join(wonders_data_path, wonder, file))

In [ ]:
wonders_df = pd.DataFrame({
    "name": wonders,
    "wonder": wonders_files
})

wonders_df.head()

In [ ]:
wonders_df.shape

# Neural Style Transfer

In [ ]:
model = hub.load("https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2")

In [ ]:
def neural_style(model, wonders_df, art_df):
    content_name, content_img = wonders_df.sample()[["name", "wonder"]].values[0]
    style_name, style_img = art_df.sample()[["artist", "art"]].values[0]
    
    content_img = Image.open(content_img)    
    content_image = tf.convert_to_tensor(content_img, np.float32)
    content_image = content_image[tf.newaxis, ...] 
    content_image /= 255.
    
    style_img = Image.open(style_img).resize((256, 256))
    style_image = tf.convert_to_tensor(style_img, np.float32)
    style_image = style_image[tf.newaxis, ...]
    style_image /= 255.

    output = model(content_image, style_image)
    result = Image.fromarray(np.uint8(output[0][0] * 255))

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(content_img)
    plt.title(content_name)
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.imshow(style_img)
    plt.title(style_name)
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.imshow(result)
    plt.title("Result")
    plt.axis("off")
    
    plt.show()

In [ ]:
neural_style(model, wonders_df, art_df)